In [0]:
%sql
USE workspace.default;
DROP TABLE IF EXISTS churn_limpo;
DROP TABLE IF EXISTS churn_silver

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.churn_zero.churn_silver AS
SELECT 
    *,
    
    -- 1. Sentimento
    ai_analyze_sentiment(CustomerFeedback) as sentiment_label,
    
    -- 2. Tópico (Labels ajustados para precisão)
    ai_classify(
        CustomerFeedback, 
        ARRAY(
            'high_price',              -- Custos/Preço alto
            'service_quality',         -- Qualidade técnica/sinal
            'internet_connection',     -- Sem internet/Lentidão
            'billing_issues',          -- Cobrança errada
            'competitor_offer',        -- Competição
            'contract_terms',          -- Problemas com contrato/Fidelidade
            'personal_reasons',        -- Mudança de casa/Circunstâncias pessoais
            'customer_service',        -- Atendimento ruim
            'general_satisfaction',    -- Elogios/Neutro (IMPORTANTE para falsos positivos)
            'other'
        )
    ) as feedback_topic,

    -- 3. Score Numérico
    CASE 
        WHEN ai_analyze_sentiment(CustomerFeedback) = 'positive' THEN 1 
        ELSE 0 
    END as sentiment_score

FROM workspace.churn_zero.churn_limpo;

In [0]:
%sql
WITH Comparacao AS (
    SELECT 
        customerID,
        CustomerFeedback,
        InternetService,
        
        -- O Velho (que já está na tabela)
        feedback_topic AS OLD_ai_classify,
        
        -- O Novo (Calculado agora com seu prompt ajustado)
        ai_gen(
            'Atue como um Analista de Retenção Sênior. Classifique o feedback em UMA categoria: 
            [sales_opportunity, high_price, internet_connection, billing_issues, competitor_offer, contract_terms, personal_reasons, customer_service, sales_misunderstanding, general_satisfaction, service_quality, other].

            PERFIL:
            - Feedback: "' || CustomerFeedback || '"
            - Tem Internet: ' || InternetService || '
            
            REGRAS OBRIGATÓRIAS:
            1. OPORTUNIDADE: Se "Tem Internet" for "No" E o cliente disser "wish that internet was included", "wish they offered internet" ou "using different provider", classifique como "sales_opportunity", a nao ser que ele fale sobre algum outro problema grave.
            2. TÉCNICO: Se falar de "slow", "speed", "outage", "reliability", USE "internet_connection". NÃO use "service_quality" genericamente.
            3. VENDA ERRADA: Se "Tem Internet" for "No" mas ele reclamar que a conexão dele (da nossa empresa) está lenta, use "sales_misunderstanding".
            4. PREÇO: "Caro" = high_price. "Concorrente melhor" = competitor_offer.
            
            SAÍDA: Apenas o nome da categoria.'
        ) AS NEW_ai_gen

    FROM workspace.churn_zero.churn_silver
    LIMIT 250
)

-- Aqui está o filtro que você queria:
SELECT * 
FROM Comparacao
WHERE OLD_ai_classify != NEW_ai_gen;

In [0]:
%sql
-- Validação da Distribuição dos Tópicos
SELECT 
    feedback_topic,
    count(*) as total
FROM workspace.churn_zero.churn_silver
WHERE churn = 1 AND sentiment_score = 1
GROUP BY feedback_topic
ORDER BY total DESC;

In [0]:
%sql
SELECT
 *
FROM workspace.churn_zero.churn_silver

In [0]:
df = spark.sql("SELECT * FROM workspace.churn_zero.churn_silver")
df.toPandas().to_csv("/Workspace/Users/joaovab@al.insper.edu.br/Hackathon_Terra_Signal_grupo_1/churn_silver.csv")